In [13]:
import re
import xml.etree.ElementTree as ET
import pandas as pd
import os

In [71]:
df = pd.DataFrame(columns=['shoot_title', 'channel', 'number', 'basename', 'mp4', 'jpg'])
df

,shoot_title,channel,number,basename,mp4,jpg


In [10]:
element_path = os.path.dirname(os.path.abspath(''))
kink = os.path.join(element_path, 'kink')
kink

'/media/gmagostini/Elements/kink'

In [12]:
shoo_number = [
    name for name in os.listdir(kink)
    if os.path.isdir(os.path.join(kink, name)) and re.match(r"^\d+$", name)
]
removed = [
    name for name in os.listdir(kink)
    if os.path.isdir(os.path.join(kink, name)) and not re.match(r"^\d+$", name)
]
removed

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'others',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [54]:
def get_files_dict(directory, code):
    files_dict = {"jpg": None, "mp4": None, "nfo": None, 'code': code}
    for file in os.listdir(directory):
        if file.endswith(".jpg"):
            files_dict["jpg"] = os.path.join(directory, file)
        elif file.endswith(".mp4"):
            files_dict["mp4"] = os.path.join(directory, file)
        elif file.endswith(".nfo"):
            nfo_path = os.path.join(directory, file)
            files_dict["nfo"] = nfo_path
            pattern = re.compile(f"^{code} - \\d+p\\.nfo$")
            files_dict["clear"] = bool(pattern.match(file))
            files_dict["base"] = file.split('.')[0]
    return files_dict


files = []
for code in shoo_number:
    _res = get_files_dict(os.path.join(kink, code), code)
    files.append(_res)

cleared = [f for f in files if f['clear'] == True]
len(cleared), cleared

(1496,
 [{'jpg': '/media/gmagostini/Elements/kink/3762/3762 - 480p.jpg',
   'mp4': '/media/gmagostini/Elements/kink/3762/3762 - 480p.mp4',
   'nfo': '/media/gmagostini/Elements/kink/3762/3762 - 480p.nfo',
   'code': '3762',
   'clear': True,
   'base': '3762 - 480p'},
  {'jpg': None,
   'mp4': '/media/gmagostini/Elements/kink/9772/9772 - 720p.mp4',
   'nfo': '/media/gmagostini/Elements/kink/9772/9772 - 720p.nfo',
   'code': '9772',
   'clear': True,
   'base': '9772 - 720p'},
  {'jpg': None,
   'mp4': '/media/gmagostini/Elements/kink/9773/9773 - 720p.mp4',
   'nfo': '/media/gmagostini/Elements/kink/9773/9773 - 720p.nfo',
   'code': '9773',
   'clear': True,
   'base': '9773 - 720p'},
  {'jpg': None,
   'mp4': '/media/gmagostini/Elements/kink/9774/9774 - 720p.mp4',
   'nfo': '/media/gmagostini/Elements/kink/9774/9774 - 720p.nfo',
   'code': '9774',
   'clear': True,
   'base': '9774 - 720p'},
  {'jpg': None,
   'mp4': '/media/gmagostini/Elements/kink/9775/9775 - 720p.mp4',
   'nfo': '/m

In [72]:
for file in files:

    tree = ET.parse(file['nfo'])
    root = tree.getroot()

    title = root.find('title').text if root.find('title') is not None else None
    channel = root.find('genre').text if root.find('genre') is not None else None

    new_row = pd.Series(data={
        'shoot_title': title,
        'channel': channel,
        'number': file['code'],
        'basename': file["base"],
        'mp4': bool(file["mp4"]),
        'jpg': bool(file["jpg"]),
    })
    df.loc[len(df)] = new_row
df

,shoot_title,channel,number,basename,mp4,jpg
0,Fuckingmachines on the Road,fucking-machines,3762,3762 - 480p,True,True
1,Pig tails + Thigh Highs + Cotton Panties + Bal...,hogtied,9772,9772 - 720p,True,False
2,PIG TAILS and THIGH HIGHS,hogtied,9773,9773 - 720p,True,False
3,WHIP OR TICKLE Jessie hates being tickled more...,hogtied,9774,9774 - 720p,True,False
4,"Pigtailed girl with thigh highs is bound, made...",hogtied,9775,9775 - 720p,True,False
...,...,...,...,...,...,...
1491,WIREDPUSSY IN NEW YORK Part 3 Wiredpussy prese...,wired-pussy,3800,3800 - 720p,True,True
1492,Nadia Styles has an Appetite for Suffering,hogtied,38015,38015 - 720p,True,False
1493,California Girl Turned Into a California Slut ...,hogtied,38017,38017 - 720p,True,False
1494,Into the Darkness,device-bondage,38018,38018 - 720p,True,False


In [73]:
df.to_csv(os.path.join(kink, 'shoots.csv'), index=False)